In [23]:
from langchain_openai import ChatOpenAI

# Functions
def multiply(a:int, b:int):
    """Mutliply"""
    return a*b

def add(a:int, b:int):
    """Addition"""
    return a+b

tools = [multiply, add]

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools)

In [24]:
from langgraph.graph import MessagesState
from langgraph.graph import START, END, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Graph
builder = StateGraph(MessagesState)

# Define nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
        tools_condition,
)
builder.add_edge("tools", "assistant")

memory = MemorySaver()
graph = builder.compile(checkpointer=MemorySaver())

In [25]:
from pprint import pprint
config = {"configurable":{"thread_id":"1"}}

input= {"messages": "Multiply 4*4"}

for event in graph.stream(input, config, stream_mode="values"):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

Multiply 4*4
================================== Ai Message ==================================
Tool Calls:
  multiply (call_eHnWfAesprcXnCi7eIrdiOP5)
 Call ID: call_eHnWfAesprcXnCi7eIrdiOP5
  Args:
    a: 4
    b: 4
================================= Tool Message =================================
Name: multiply

16
================================== Ai Message ==================================

The result of multiplying 4 by 4 is 16.


In [26]:
graph.get_state({'configurable': {'thread_id': '1'}})

StateSnapshot(values={'messages': [HumanMessage(content='Multiply 4*4', additional_kwargs={}, response_metadata={}, id='2ec56e41-37be-4fd9-83a4-0c609b135526'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eHnWfAesprcXnCi7eIrdiOP5', 'function': {'arguments': '{"a":4,"b":4}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 83, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-585562b1-6205-4e48-b1c1-c41a4557743d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 4, 'b': 4}, 'id': 'call_eHnWfAesprcXnCi7eIrdiOP5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 8

In [27]:
all_states = [s for s in graph.get_state_history(config)]
to_fork  = all_states[-2]
to_fork.values["messages"][0].id

'2ec56e41-37be-4fd9-83a4-0c609b135526'

In [28]:
fork_config = graph.update_state(
    to_fork.config,
    {"messages":[HumanMessage("multiply 5*5", id = to_fork.values["messages"][0].id)]} # Invoke reducer function to overwrite the pervious query!
)
fork_config

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1efedec4-c01c-631b-8001-79102f9b7319'}}

In [29]:
all_states = [s for s in graph.get_state_history(config)]
all_states

[StateSnapshot(values={'messages': [HumanMessage(content='multiply 5*5', additional_kwargs={}, response_metadata={}, id='2ec56e41-37be-4fd9-83a4-0c609b135526')]}, next=('assistant',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efedec4-c01c-631b-8001-79102f9b7319'}}, metadata={'source': 'update', 'writes': {'__start__': {'messages': [HumanMessage(content='multiply 5*5', additional_kwargs={}, response_metadata={}, id='2ec56e41-37be-4fd9-83a4-0c609b135526')]}}, 'thread_id': '1', 'step': 1, 'parents': {}, 'checkpoint_ns': '', 'checkpoint_id': '1efedec3-889a-6000-8000-271e06fb40c5'}, created_at='2025-02-18T11:34:21.502299+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efedec3-889a-6000-8000-271e06fb40c5'}}, tasks=(PregelTask(id='6393ef18-0163-2e72-717e-e82d8308d2ad', name='assistant', path=('__pregel_pull', 'assistant'), error=None, interrupts=(), state=None, result=None),)),
 StateSnapshot(values={'m

In [ ]:
for event in graph.stream(None, fork_config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

multiply 5*5
================================== Ai Message ==================================
Tool Calls:
  multiply (call_paxMrGjB4Ur0GVfPmxENDJZA)
 Call ID: call_paxMrGjB4Ur0GVfPmxENDJZA
  Args:
    a: 5
    b: 5
================================= Tool Message =================================
Name: multiply

25


In [ ]:
graph.get_state({'configurable': {'thread_id': '1'}})

StateSnapshot(values={'messages': [HumanMessage(content='multiply 5*5', additional_kwargs={}, response_metadata={}, id='527a9b39-ea1c-4151-980a-aa74cc0b988a'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EeDoPBzxoA5n4grsJ96dBwPJ', 'function': {'arguments': '{"a":5,"b":5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 83, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-85c0a8d1-9caa-4322-95ee-8209297bb0dc-0', tool_calls=[{'name': 'multiply', 'args': {'a': 5, 'b': 5}, 'id': 'call_EeDoPBzxoA5n4grsJ96dBwPJ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 8